In this session (held on 9-8-2023), we started working with Anaconda on our own machines. We had to setup a virtual environment, which took some time, and then started looking at two libraries for working with tabular data in python, namely, `pandas` and `polars`. The following code block imports the libraries.

In [1]:
import pandas as pd
import polars as pl

You can see what version of a library you are running by accessing the `.__version__` attribute of an imported library. The following code block shows how we can pront the version of the `polars` library that we are using. Although your version may differ from the one shown, you should note that the code was developed using the version shown and you may run into some differences, even errors, if your version differs.

In [2]:
pl.__version__

'0.19.2'

The following code block shows the version of `pandas` library that is being used.

In [3]:
pd.__version__

'2.0.3'

We will now look at a few differences between the two imported libraries in terms of computational speed, memory usage, and syntax. We will start by looking at the computational efficiency of data ingestion. The following code block uses the `%%timeit` *magic* method available in Jupyter to estimate the average time required for `pandas` to load the data contained in the `churn.csv` file using the `pandas` `read_csv` method. **Note**: this code expects that the `churn.csv` file is located in the same directory as this notebook.

In [4]:
%%timeit

pd.read_csv('churn.csv')

20.6 ms ± 2.83 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


The following code block uses the `%%timeit` *magic* method to estimate the average time required for `polars` to load the data contained in the `churn.csv` file using the `polars` `read_csv` method. Although times will differ based on the machine the code is ran on, the time required for `polars` should be significantly lower than that required for `pandas`. 

In [5]:
%%timeit

pl.read_csv('churn.csv')

2.44 ms ± 22.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The following code block reads the data and stores it two objects: 1) a `pandas` `DataFrame` named `data_pd` and 2) a `polars` `DataFrame` named `data_pl`. 

In [6]:
data_pd = pd.read_csv('churn.csv')
data_pl = pl.read_csv('churn.csv')

The following code block uses the `memory_usage` method of a `pandas` `DataFrame` to estimate the amount of memory occupied by the `data_pd` object in bytes.

In [7]:
data_pd.memory_usage(deep=True).sum()

8166479

The following code block uses the `estimated_size` method of a `polars` `DataFrame` to estimate the amount of memory occupied by the `data_pl` object in bytes. We observe that the `polars` `DataFrame` occupies significantly less memory than the `pandas` `DataFrame`, even though it is storing the same data. **In general, `polars` tends to be significantly more efficient than `pandas` with respect to both computational time and memory usage. This is an artifact of the way that it interacts with system resources and its usage of the Apache Arrow Columnar specification for data storage (see https://arrow.apache.org/docs/format/Columnar.html for more information).

In [8]:
data_pl.estimated_size()

1914043

A very common task when working with data is a `groupby-aggregation` sequence where some operation is performed across groups of data. The following code block shows such an aggregation using the `pandas` object. Specifically, we are grouping on unique values in the `Partner` column and getting the number of unique values for the `customerID` column for each of the `Partner` groups. **Note**: recall that you could run `data_pd.head()` to see the first five rows of the `pandas` `DataFrame`, which would include the column labels.

In [9]:
data_pd.groupby('Partner').agg(
    unique_customers=('customerID', 'nunique')
)

,unique_customers
Partner,
No,3641
Yes,3402


The following code block performs the same operation on the `polars` object. Notice that there are some differences in the syntax. First, instead of `groupby`, `polars` uses `group_by`. Also, the way that we specify the column aggregation and its label is different (i.e., `unique_customers=('customerID', 'nunique')` in `pandas` **vs.** `pl.col('customerID').n_unique().alias('unique_customers')` in `polars`).

In [10]:
data_pl.group_by('Partner').agg(
    pl.col('customerID').n_unique().alias('unique_customers')
)

Partner,unique_customers
str,u32
"""No""",3641
"""Yes""",3402


The following code block times a more complex filtering operation in `pandas`. Specifically, we:
1. define a mask that determines rows where the value in the `tenure` column is greater than or equal to 12 (`tenure_mask`),
2. define a mask that determines rows where the value in the `PhoneService` column is `Yes` (`phone_service_mask`),
3. combines the `tenure_mask` and `phone_service_mask` objects to get a mask for rows where the value in the `tenure` column is greater than or equal to 12 **and** the value in the `PhoneService` column is `Yes` (`combined_mask`), and
4. gets the number of unique customer IDs by value in the `Partner` column for the data filtered by the `combined_mask`.

In [11]:
%%timeit

tenure_mask = data_pd['tenure'] >= 12
phone_service_mask = data_pd['PhoneService'] == 'Yes'
combined_mask = tenure_mask & phone_service_mask
data_pd[combined_mask].groupby('Partner').agg(
    unique_customers=('customerID', 'nunique')
)

4.14 ms ± 63.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The following code block performs the same sequence of operations using the `polars` object. Note that when we used `pandas`, we created several temporary objects along the way to completing our task. In `polars`, we can specify a more *functional* sequence of operations, where method calls are *chained* together.

In [12]:
%%timeit

data_pl.filter(
    pl.col('tenure') >= 12
).filter(
    pl.col('PhoneService') == 'Yes'
).group_by('Partner').agg(
    pl.col('customerID').n_unique().alias('unique_customers')
)

1.88 ms ± 276 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In our previous `pandas` and `polars` implementations, we (the user) was controlling the order of operations on the data by the sequence we applied them. However, there has been significant research effort put into the development of *optimizers* that can determine the best sequence of operations for data manipulations that can significantly reduce computational time. The `polars` library has such an optimizer built in. To let `polars` use its optimizer, we can specify that we want the sequence of operations evaluated in a `lazy` fashion. By doing so, we are telling `polars` what we want to do, but letting the library figure out the best way to accomplish it. The following code block times such a `lazy` implementation of the previous code block. Note the `.lazy()` call towards the beginning and the `.collect()` call at the end.

In [13]:
%%timeit

data_pl.lazy().filter(
    pl.col('tenure') >= 12
).filter(
    pl.col('PhoneService') == 'Yes'
).group_by('Partner').agg(
    pl.col('customerID').n_unique().alias('unique_customers')
).collect()

701 µs ± 60.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
